In [1]:
import pandas as pd
from prophet import Prophet
# from neuralprophet import NeuralProphet# as Prophet
# from neuralprophet import set_log_level
# set_log_level("ERROR")

from dotenv import load_dotenv
from pathlib import Path
import os

# from ezekial.prophet import Prophet
import yfinance as yf
import requests
import base64
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count, freeze_support
import time
import warnings
import sys

/home/unknown/anaconda3/envs/forecastappenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Arguements
warnings.filterwarnings("ignore")
load_dotenv()
url = os.getenv("DJANGO_SERVER_URL")
weeeks_prior_data_end_date = int(sys.argv[1]) #260
forecast_days_ahead = int(sys.argv[2]) #90
ticker_path = sys.argv[3]#'data/S&P500 tickers.csv'
forecast_img_path = sys.argv[4]#'images/forecast_temp/forecast.png'

In [8]:
# # Arguements
# warnings.filterwarnings("ignore")
# load_dotenv()
# url = os.getenv("DJANGO_SERVER_URL")
# weeeks_prior_data_end_date = 260
# forecast_days_ahead = 90
# ticker_path = 'data/S&P500 tickers.csv'
# forecast_img_path = 'images/forecast_temp/forecast.png'

In [9]:
# print(weeeks_prior_data_end_date)
# print(forecast_days_ahead)
# print(ticker_path)
# print(forecast_img_path)

In [15]:
def load_tickers(ticker_path):
    """
    Takes five hundred tickers, which is max calls of yfinance for public ip.
    """
    ticker_file_path = Path(ticker_path)
    ticker_df = pd.read_csv(ticker_file_path)
    prophet_ticker_list = ticker_df['Symbol'].head(500).to_list()
    return prophet_ticker_list

def django_get(url):
    response = requests.get(url)
    response_json = response.json()
    return response_json

def clear_django_sql_db(url):
    """
    Clears the sql db in the django api.
    "DJANGO_SERVER_URL" is provided in .env file.
    """

    try:
        # load_dotenv()
        # url = os.getenv("DJANGO_SERVER_URL")

        django_api_items = django_get(url)

        if len(django_api_items) != 0:
            for x, index_id in enumerate(django_api_items):
                x += 1
                str_indx = str(index_id['id'])

                if x == 1:
                    url += str(str_indx)
                    requests.delete(url)
                    # print(f'entered if block\t{url}')
                else:
                    try:
                        url = url[:-len(str_indx)] + str_indx
                        requests.delete(url)
                        # print(f'entered else block\t{url}')
                    # For when the number of digits change
                    except:
                        url = url[:-len(str_indx) + 1] + str_indx
                        requests.delete(url)                    
                        # print(f'entered else block\t{url}')
            print(f"Django SQL DB cleared:\n\n{django_get(url)}")
        else:
            print(f"Django SQL DB Already empty:\n\n{django_get(url)}")
    except:
        print(f"Django server did not clear:\n\n{django_get(url)}")

def run_prophet(timeserie, forecast_ahead = forecast_days_ahead):
    model = Prophet(yearly_seasonality=False,daily_seasonality=False)
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=forecast_ahead, include_history=True)
    forecast = model.predict(forecast)
    return forecast

def make_plots(url):
    # Attempt to clear the SQL DB if it fails initially before uploading the data again, 
    # to reduce DB size for POST request on the APP side
    try: clear_django_sql_db(url = url)
    except: 
        try: clear_django_sql_db(url = url)
        except: error_msg = '\t##### Could not clear the Django SQL DB  #####'

    for idx, fig_plot in enumerate(predictions):
        forecast_plot = pd.merge(left=fig_plot, right=series[idx], on='ds', how='outer')
        ticker_sym = adj_close_df.columns[idx]
        forecast_plot = forecast_plot[-(forecast_days_ahead - 10) - 30:-(forecast_days_ahead - 10)]
        forecast_plot.index = forecast_plot['ds']

        with plt.style.context('dark_background'):
            fig, ax = plt.subplots()
            forecast_plot[['y','yhat']].plot(ax=ax, figsize = (4,5), style=['o-','--'],color=['r','lightblue'],linewidth=1)
            ax.set_xlabel('Date', fontdict={'size':12})
            ax.set_ylabel('Price', fontdict={'size':12})
            ax.yaxis.set_major_formatter('${x:,.0f}')
            ax.legend(['Actual','Forecasted'], title= f'{ticker_sym}', loc='upper center', frameon=False)
            # ax.set_facecolor('xkcd:black')
        # plt.show()
        try:
            fig.savefig(forecast_img_path)
            with open(forecast_img_path, "rb") as image2string:
                converted_string = base64.b64encode(image2string.read())
            image2string.close()

            json_post = {
                "ticker": f"{ticker_sym}",
                "time": f'{today}'[:15],
                "encoded_string": f"{converted_string}"
            }

            try:
                requests.post(url,json_post)
                print(f"+++ {ticker_sym} success!")
            except:
                print(f"\t--- {ticker_sym} failed ---")
        except:
            print(f"\t---# {ticker_sym} failed #---")
            print("\n\n\tyfinance API daily limit reached.")

In [ ]:
start_time = time.time()
today = datetime.today()
start_date = today - timedelta(weeks=weeeks_prior_data_end_date)
start_date = start_date.replace(second=0, microsecond=0)

data = yf.download(load_tickers(ticker_path), start=start_date)
adj_close_df = data['Adj Close']
adj_close_df = adj_close_df.dropna(axis=1, how='all')
series = [adj_close_df[f'{col}'].reset_index().rename({'Date':'ds', f'{col}':'y'}, axis=1) for col in adj_close_df]

start_time0 = time.time()
p = Pool(cpu_count())
predictions = list(tqdm(p.imap(run_prophet, series), total=len(series)))
p.close()
p.join()
print("--- %s seconds ---" % (time.time() - start_time0))

make_plots(url)

try: print(error_msg)
except: print("\t+++++ The SQL DB was successfully cleared +++++")

print("--- %s seconds ---" % (time.time() - start_time))

[*********************100%***********************]  500 of 500 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


  0%|                                                  | 0/498 [00:00<?, ?it/s]

In [12]:
# clear_django_sql_db(url)

Django SQL DB cleared:

{'detail': 'Not found.'}
